In [8]:
#importing the libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [9]:
# The URL of the webpage to scrape table data about Super Bowl Halftime Shows
url = 'https://en.wikipedia.org/wiki/List_of_Super_Bowl_halftime_shows#cite_note-68'
response = requests.get(url)
soup = BeautifulSoup(response.text, features = "html.parser")

In [10]:
# Define the get_content_value function to handle nested <li> and <a> tags and separate them by commas
def get_content_value(row_data):
    if row_data.find("li"):
        return ', '.join([li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")])
    elif row_data.find("a"):
        return ', '.join([a.get_text(" ", strip=True).replace("\xa0", " ") for a in row_data.find_all("a")])
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

# Extract the rows using the updated get_content_value function
tables = soup.find_all("table", {"class": "wikitable"})

# Preprocess the tables to ensure <li> and <a> tags are separated by commas
for table in tables:
    for tr in table.find_all('tr'):
        for td in tr.find_all('td'):
            td.string = get_content_value(td)

#process each table found            
all_dfs = [pd.read_html(str(table))[0] for table in tables]

#combine all dataframes into one
df = pd.concat(all_dfs, ignore_index=True)

print(df.tail())

     Super Bowl               Date                               Location  \
54     LV, show   February 7, 2021  Raymond James Stadium, Tampa, Florida   
55    LVI, show  February 13, 2022    SoFi Stadium, Inglewood, California   
56   LVII, show  February 12, 2023  State Farm Stadium, Glendale, Arizona   
57  LVIII, show  February 11, 2024    Allegiant Stadium, Paradise, Nevada   
58    LIX, show   February 9, 2025         Caesars Superdome, New Orleans   

   Theme Performer(s)                                           Producer  \
54   NaN          NaN  Jesse Collins, Jay-Z, Roc Nation, Diversified ...   
55   NaN          NaN  Jesse Collins, Jay-Z, Roc Nation, Diversified ...   
56   NaN          NaN  Jesse Collins, Jay-Z, Roc Nation, Diversified ...   
57   NaN          NaN  Jesse Collins, Jay-Z, Roc Nation, Diversified ...   
58   NaN          NaN  Jesse Collins, Jay-Z, Roc Nation, Diversified ...   

                                              Setlist                    Ref.  \

C:\Users\Jade Ana-Maria\AppData\Local\Temp\ipykernel_9624\3574281259.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_dfs = [pd.read_html(str(table))[0] for table in tables]
C:\Users\Jade Ana-Maria\AppData\Local\Temp\ipykernel_9624\3574281259.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_dfs = [pd.read_html(str(table))[0] for table in tables]
C:\Users\Jade Ana-Maria\AppData\Local\Temp\ipykernel_9624\3574281259.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_dfs = [pd.read_html(str(table))[0] for table in tables]
C:\Users\Jade Ana-Maria\AppData\Local\Temp\ipykernel_9624\3574281259.py:20: Fut

In [11]:
#rename superbowl column to get rid of whitespace
if "Super Bowl" in df.columns:
    df.rename(columns={"Super Bowl": "Super_Bowl"}, inplace=True)

#Preprocessing the data to remove unwanted characters

#remove the word show from the Super_Bowl column
df["Super_Bowl"] = df["Super_Bowl"].str.replace("(show)", "", regex=False).str.strip()
#remove 50 from the Super_Bowl column
df["Super_Bowl"] = df["Super_Bowl"].str.replace("50", "L", regex=False).str.strip()
print(df.head())


  Super_Bowl              Date                       Location           Theme  \
0          I  January 15, 1967  Los Angeles Memorial Coliseum               —   
1         II  January 14, 1968              Miami Orange Bowl               —   
2        III  January 12, 1969              Miami Orange Bowl  America Thanks   
3         IV  January 11, 1970    Tulane Stadium, New Orleans      Mardi Gras   
4    V, show  January 17, 1971              Miami Orange Bowl               —   

                                        Performer(s)      Producer  \
0  University of Arizona Symphonic Marching Band,...  Tommy Walker   
1                         Grambling State University             —   
2  Florida A&M University band, Miami-area high s...             —   
3  Marguerite Piazza, Doc Severinsen, Al Hirt, Li...             —   
4  Southeast Missouri State, Anita Bryant, Up Wit...             —   

                                             Setlist  \
0  " The Sound of Music ", Medley: "

In [12]:
len(df)

59

In [13]:
df.to_csv("halftime_musicians_raw.csv", index=False)
print("halftime_musicians_raw.csv'")

halftime_musicians_raw.csv'
